In [21]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary
import numpy as np

# 读取Excel文件中的数据
file1_path = '附件1.xlsx'
file2_path = '附件2_1.xlsx'

# 读取每个工作表的数据
file1_data1 = pd.read_excel(file1_path, sheet_name='乡村的现有耕地')
file1_data2 = pd.read_excel(file1_path, sheet_name='乡村种植的农作物')
file2_data1 = pd.read_excel(file2_path, sheet_name='Sheet1')
file2_data2 = pd.read_excel(file2_path, sheet_name='Sheet2')

# 清理列名
file1_data1.columns = ['land_name', 'land_type', 'land_area', 'description']
file1_data2.columns = ['crop_id', 'crop_name', 'crop_type', 'suitable_land', 'description']
file2_data1.columns = ['land_name', 'crop_id', 'crop_name', 'crop_type', 'planting_area', 
                      'planting_season', 'land_type', 'yield_per_mu', 'planting_period', 'expected_sales']
file2_data2.columns = ['serial_no', 'crop_id', 'crop_name', 'land_type', 'planting_season', 
                       'yield_per_mu', 'planting_cost_per_mu', 'sales_price_per_unit']

crop_type_mapping = file1_data2[['crop_id', 'crop_type']].set_index('crop_id').to_dict()['crop_type']
file2_data2['crop_type'] = file2_data2['crop_id'].map(crop_type_mapping)

# 将价格区间转换为均值
def parse_price_range(price_str):
    if isinstance(price_str, str) and '-' in price_str:
        low, high = map(float, price_str.split('-'))
        return (low + high) / 2
    return float(price_str)

file2_data2['average_price'] = file2_data2['sales_price_per_unit'].apply(parse_price_range)

# 创建作物价格、产量、销售量、成本字典
crop_prices = dict(zip(file2_data2['crop_id'], file2_data2['average_price']))
crop_yields = dict(zip(file2_data1['crop_id'], file2_data1['yield_per_mu']))
expected_sales = dict(zip(file2_data1['crop_id'], file2_data1['expected_sales']))
cost = dict(zip(file2_data2['crop_id'], file2_data2['planting_cost_per_mu']))

# 创建土地面积的字典
land_areas = dict(zip(file1_data1['land_name'], file1_data1['land_area']))

# 定义每个地块类型对应的种植季节数
season_info = {
    '平旱地': 1,  # 每年只能种一季
    '梯田': 1,    # 每年只能种一季
    '山坡地': 1,  # 每年只能种一季
    '水浇地': 2,  # 每年可以种一季或两季
    '大棚': 2     # 每年可以种两季
}

# 根据 land_type 决定每块地每年可以种的季节数
seasons = {row['land_name']: season_info.get(row['land_type'], 1) for idx, row in file1_data1.iterrows()}

# 创建一个字典，存储每个作物可以种植的土地类型
crop_suitable_land = {}
for idx, row in file1_data2.iterrows():
    if pd.notna(row['suitable_land']):
        suitable_land_types = row['suitable_land'].split('\n')
        crop_suitable_land[row['crop_id']] = suitable_land_types

# 定义年份范围
years = list(range(2024, 2031))  # 从2024年到2030年，共7年

# 定义不确定性的范围（每个情景会根据这些不确定性生成不同的参数）
price_growth_rate = {
    '粮食': 1.00,  # 粮食价格稳定
    '蔬菜': 1.05,  # 蔬菜价格年增长5%
    '食用菌': lambda: np.random.uniform(0.95, 0.99)  # 食用菌价格每年下降1%-5%
}

sales_growth_rate = {
    '小麦': lambda: np.random.uniform(1.05, 1.10),  # 小麦销售量年增长5%-10%
    '玉米': lambda: np.random.uniform(1.05, 1.10),  # 玉米销售量年增长5%-10%
    '其他': lambda: np.random.uniform(0.95, 1.05)  # 其他作物销售量波动±5%
}

yield_variation = lambda: np.random.uniform(0.90, 1.10)  # 亩产量波动±10%
cost_growth_rate = 1.05  # 成本年增长5%

# 创建情景
def generate_scenarios(num_scenarios):
    scenarios = []
    for _ in range(num_scenarios):
        scenario = {}
        for crop_id, crop_name in zip(file2_data2['crop_id'], file2_data2['crop_name']):
            crop_type = file2_data2[file2_data2['crop_id'] == crop_id]['crop_type'].values[0]
            # 随机生成不同情景下的价格、产量、成本、销售量
            if crop_type == '粮食':
                price = crop_prices[crop_id] * price_growth_rate['粮食']
            elif crop_type == '蔬菜':
                price = crop_prices[crop_id] * price_growth_rate['蔬菜']
            else:
                price = crop_prices[crop_id] * price_growth_rate['食用菌']()
            
            sales = expected_sales[crop_id] * sales_growth_rate.get(crop_name, sales_growth_rate['其他'])()
            yield_mu = crop_yields[crop_id] * yield_variation()
            cost_mu = cost[crop_id] * cost_growth_rate
            
            scenario[crop_id] = {
                'price': price,
                'sales': sales,
                'yield': yield_mu,
                'cost': cost_mu
            }
        scenarios.append(scenario)
    return scenarios

# 生成10个情景
scenarios = generate_scenarios(10)

# 定义优化模型
model = LpProblem("Crop_Allocation_Optimization_Scenario", LpMaximize)

# 决策变量
x = LpVariable.dicts("plant_area", 
                     [(i, j, k, t) for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)], 
                     lowBound=0)  # 种植面积不能为负

# 二值变量 y 表示是否种植该作物
y = LpVariable.dicts("plant_decision", 
                     [(i, j, k, t) for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)], 
                     cat=LpBinary)  # y 变量是二进制变量

# 目标函数：考虑多个情景下的净收益
model += lpSum(
    lpSum(scenario[j]['price'] * scenario[j]['yield'] * x[i, j, k, t] - scenario[j]['cost'] * x[i, j, k, t]
          for i in land_areas for j in crop_yields for t in range(1, seasons[i] + 1))
    for scenario in scenarios for k in years
)

# 约束1：耕地面积约束 - 调整每块地每年每季种植面积的约束，确保每年不超过可用面积
for i in land_areas:
    for k in years:
        # 每年总种植面积不能超过该地块的总可用面积
        model += lpSum(x[i, j, k, t] for j in crop_yields for t in range(1, seasons[i] + 1)) <= land_areas[i]

# 手动输入豆类作物的 crop_id 列表
bean_crops = [1, 2, 3, 4, 5, 17, 18, 19]

# 约束2：每三年内至少种植一次豆类作物
for i in land_areas:
    # 确保每块地在每三年内至少种植一次豆类作物
    for k_start in range(2024, 2031, 3):  # 从2024年开始，步长为3年
        model += lpSum(y[i, j, k, t] for j in bean_crops for k in range(k_start, min(k_start+3, 2031))
                       for t in range(1, seasons[i] + 1)) >= 1

# 约束3：作物产量不得超过预期销售量
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                model += crop_yields[j] * x[i, j, k, t] <= expected_sales[j]

# 约束4：同一地块不能在连续季节种植相同作物
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i]):  # 检查同一年不同季节
                model += y[i, j, k, t] + y[i, j, k, t+1] <= 1

# 约束5：最小种植面积约束（当 y[i, j, k, t] == 1，即种植某种作物时，才要求种植面积大于最小面积）
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                # 如果种植某作物 (y[i, j, k, t] == 1)，种植面积必须 >= 0.5 * land_areas[i]
                model += x[i, j, k, t] >= 0.5 * land_areas[i] * y[i, j, k, t]

# 新增约束6：作物只能种植在适合的土地类型上
for i in land_areas:
    for j in crop_yields:
        land_type = file1_data1.loc[file1_data1['land_name'] == i, 'land_type'].values[0]
        # 先检查 crop_id 是否存在于 crop_suitable_land 字典中
        if j in crop_suitable_land:
            # 如果当前地块类型不适合该作物种植，则不允许种植
            if land_type not in crop_suitable_land[j]:
                for k in years:
                    for t in range(1, seasons[i] + 1):
                        model += x[i, j, k, t] == 0  # 不允许种植不适合的作物

# 求解模型
model.solve()

# 逐年计算目标函数值（考虑多个情景）
for k in years:
    yearly_obj_value = lpSum(
        lpSum(
            (scenario[j]['price'] * scenario[j]['yield'] - scenario[j]['cost']) * x[i, j, k, t].varValue
            for i in land_areas for j in crop_yields for t in range(1, seasons[i] + 1)
            if x[i, j, k, t].varValue > 0  # 只计算种植面积大于 0 的决策变量
        ) for scenario in scenarios
    )
    print(f"Year {k} 目标函数值（多个情景的平均收益）: {yearly_obj_value}")

# 按年整理种植策略
yearly_planting_strategy = {}

# 遍历每一个决策变量，按年（k）进行汇总
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                if x[i, j, k, t].varValue > 0:  # 只输出非零种植面积的策略
                    if k not in yearly_planting_strategy:
                        yearly_planting_strategy[k] = []
                    yearly_planting_strategy[k].append({
                        'land': i,
                        'crop_id': j,
                        'season': t,
                        'planting_area': x[i, j, k, t].varValue
                    })

# 按年份输出种植策略
for year, strategy in yearly_planting_strategy.items():
    print(f"Year {year}:")
    df_yearly = pd.DataFrame(strategy)
    print(df_yearly)

Year 2024 目标函数值（多个情景的平均收益）: 213145706.13399023
Year 2025 目标函数值（多个情景的平均收益）: 205612752.0383625
Year 2026 目标函数值（多个情景的平均收益）: 202759595.07274732
Year 2027 目标函数值（多个情景的平均收益）: 209028798.06415877
Year 2028 目标函数值（多个情景的平均收益）: 204388612.95889872
Year 2029 目标函数值（多个情景的平均收益）: 208100642.22204256
Year 2030 目标函数值（多个情景的平均收益）: 185076235.15315852
Year 2025:
    land  crop_id  season  planting_area
0     A1        2       1           40.0
1     A1       20       1           10.5
2     A1       36       1           10.0
3     A1       28       1            0.3
4     A1       21       1            0.3
..   ...      ...     ...            ...
828   F1       40       1            0.3
829   F2       40       1            0.6
830   F3       40       1            0.6
831   F4       19       1            0.3
832   F4       40       1            0.3

[833 rows x 4 columns]
Year 2027:
    land  crop_id  season  planting_area
0     A1        2       1           40.0
1     A1       20       1           10.5
2     A1   

In [22]:
import pandas as pd

# 读取用户上传的 result1_1.xlsx
result_file_path = 'result2.xlsx'
result_data_template = pd.read_excel(result_file_path)

# 定义作物编号与名称的映射（使用正确的列名）
crop_id_to_name = dict(zip(file1_data2['crop_id'], file1_data2['crop_name']))

# 创建一个函数来根据种植策略更新表格
def update_result_table(planting_strategy, result_df):
    for record in planting_strategy:
        land = record['land']  # 地块名称
        crop_id = record['crop_id']  # 作物编号
        season = record['season']  # 季节
        planting_area = record['planting_area']  # 种植面积
        
        # 将 crop_id 转换为 crop_name
        crop_name = crop_id_to_name.get(crop_id, None)
        
        # 找到地块对应的行
        row_index = result_df.index[result_df['地块名'] == land]
        if season == 1:
               # 将种植面积写入第一季的列
            result_df.loc[row_index, crop_name] = planting_area
        else:
                # 将种植面积写入第二季的列
            result_df.loc[row_index + 1, crop_name] = planting_area

# 创建一个字典存储每年的结果
yearly_result_data = {}

# 遍历每个年份，将每年的种植策略填入对应的表格
for year, strategy in yearly_planting_strategy.items():
    # 使用 result_data_template 作为基础
    result_data_year = result_data_template.copy()
    # 更新每年的数据
    update_result_table(strategy, result_data_year)
    # 将结果保存到字典中
    yearly_result_data[f'{year}'] = result_data_year

# 将所有年份的结果写入一个包含多个 sheet 的 Excel 文件
with pd.ExcelWriter('result2_multisheet.xlsx') as writer:
    for sheet_name, data in yearly_result_data.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

print('成功')

C:\Users\cyk\AppData\Roaming\Python\Python38\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


成功
